In [ ]:
cd ~/plp-official-tmlr2024/

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import torch 
from tqdm import tqdm
from utils import fix_random_seeds
from torch.utils.data import Subset, DataLoader,Dataset
from torch.nn import functional as F
from torchmetrics.functional import image_gradients

from eval_utils import roc_auc_score, OOD_classifier_knn
from loaders import get_dataset
from model_builders.model_utils import split_normalization
from model_builders import *

out_dir = Path('./experiments/adversarial-dataset')
out_dir.mkdir(exist_ok=True)
fix_random_seeds(0)
available_models('openclip*')
model, preprocess = load_backbone('openclip_ViT-bigG-14/laion2b')
for p in model.parameters():
    p.requires_grad = False

model.cuda().eval()
pre, norm = split_normalization(preprocess)

In [ ]:
N = 1000
smooth_lambda = 5e3
steps= 250

cifar100_train = get_dataset('CIFAR100', transform=pre, train=True)
train_idx = np.random.choice(len(cifar100_train), N)
cifar100_train = Subset(cifar100_train, train_idx)

cifar100_test = get_dataset('CIFAR100', transform=pre, train=False)
cifar100_test = Subset(cifar100_test, np.random.choice(len(cifar100_test), N))

cifar10 = get_dataset('CIFAR10', transform=pre, train=False)
cifar10 = Subset(cifar10, np.random.choice(len(cifar10), N))

In [ ]:
def compute_adv(orig_dset, target_dset, lr=1e-3, steps=200, eps=1e-3, smooth_lambda=1e3, bs=16):
    perts = []
    ys = []
    y_embed = []
    adv_embed = []

    loader_x = DataLoader(target_dset, batch_size=bs, pin_memory=True, shuffle=False)
    loader_y = DataLoader(orig_dset, batch_size=bs, pin_memory=True, shuffle=False)
    for (x,_), (y,_) in tqdm(zip(loader_x, loader_y), total=len(loader_y)):
        ys.append(y)
        with torch.no_grad():
            target = model(norm(x.cuda()))
        y = y.cuda()
        pert = torch.randn_like(y, requires_grad=True)
        pert.data *= eps
        pert.data.clamp_(-y, 1-y)
        opt = torch.optim.Adam([pert], lr=lr)
        with tqdm(range(steps), leave=False) as pbar:
            for _ in pbar:
                pert.grad = None
                out = model(norm(y + pert))
                dy, dx = image_gradients(pert)
                smooth_loss = dy.square().mean() + dx.square().mean()
                sim = F.cosine_similarity(out, target).mean()
                loss = -sim + smooth_lambda * smooth_loss
                loss.backward()
                opt.step()
                
                pert.data.clamp_(-y, 1-y)
                pbar.set_postfix(sim=sim.item(), norm=pert.square().mean().item())
        perts.append(pert.detach().cpu())
        with torch.no_grad():
            y_embed.append(model(norm(y)).cpu())
            adv_embed.append(model(norm(y + pert)).cpu())

    ys = torch.cat(ys)
    perts = torch.cat(perts)
    y_embed = torch.cat(y_embed)
    adv_embed = torch.cat(adv_embed)
    return dict(
        orig=ys,
        orig_embed=y_embed,
        perturbations=perts,
        adv_embed=adv_embed)

In [ ]:
result_smooth = compute_adv(cifar10, cifar100_train, bs=16, smooth_lambda=smooth_lambda, steps=steps)
result_patchy = compute_adv(cifar10, cifar100_train, bs=16, smooth_lambda=0, steps=steps)
torch.save(result_smooth, out_dir / 'perts_smooth.pt')
torch.save(result_patchy, out_dir / 'result_patchy.pt')
torch.save(train_idx, out_dir / 'target_idx.pt')
targets, _ = next(iter(DataLoader(cifar100_train, N)))
torch.save(targets, out_dir / 'targets.pt')

In [ ]:
x_embed = []
loader = DataLoader(cifar100_test, batch_size=32, pin_memory=True)
with torch.no_grad():
    for x, _ in tqdm(loader):
        x = x.cuda()
        x_embed.append(model(norm(x)).cpu())
x_embed = torch.cat(x_embed)
x_train_embed = load_embeds(arch='openclip_ViT-bigG-14/laion2b', dataset='CIFAR100', test=False)

In [ ]:
def eval_auroc(result, x_embed, train_idx):
    x_train_embed = load_embeds(arch='openclip_ViT-bigG-14/laion2b', dataset='CIFAR100', test=False)
    y_embed = result['orig_embed']
    adv_embed = result['adv_embed']
    
    bs=8
    scores_in = OOD_classifier_knn(x_train_embed, x_embed, k=1, metric='cos-sim', args=None, num_chunks=bs)
    scores_out = OOD_classifier_knn(x_train_embed, y_embed, k=1, metric='cos-sim', args=None, num_chunks=bs)
    scores_adv = OOD_classifier_knn(x_train_embed, adv_embed, k=1, metric='cos-sim', args=None, num_chunks=bs)
    labels = torch.cat((torch.ones_like(scores_in), torch.zeros_like(scores_out)))
    
    auroc_real1 = roc_auc_score(labels.cpu(), torch.cat((scores_in, scores_out)).cpu()) * 100
    auroc_adv1 = roc_auc_score(labels.cpu(), torch.cat((scores_in, scores_adv)).cpu()) * 100
    
    mask = torch.ones(len(x_train_embed)).bool()
    mask[train_idx] = False
    x_train_embed = x_train_embed[mask]
    
    scores_in = OOD_classifier_knn(x_train_embed, x_embed, k=1, metric='cos-sim', args=None, num_chunks=bs)
    scores_out = OOD_classifier_knn(x_train_embed, y_embed, k=1, metric='cos-sim', args=None, num_chunks=bs)
    scores_adv = OOD_classifier_knn(x_train_embed, adv_embed, k=1, metric='cos-sim', args=None, num_chunks=bs)
    labels = torch.cat((torch.ones_like(scores_in), torch.zeros_like(scores_out)))
                        
    auroc_real2 = roc_auc_score(labels.cpu(), torch.cat((scores_in, scores_out)).cpu()) * 100
    auroc_adv2 = roc_auc_score(labels.cpu(), torch.cat((scores_in, scores_adv)).cpu()) * 100
                        
    return dict(
        auroc_real=auroc_real1,
        auroc_adv=auroc_adv1,
        auroc_real_sub=auroc_real2,
        auroc_adv_sub=auroc_adv2)

In [ ]:
smooth_auroc = eval_auroc(result_smooth, x_embed, train_idx)
patchy_auroc = eval_auroc(result_patchy, x_embed, train_idx)
smooth_auroc['smooth_lambda'] = smooth_lambda
patchy_auroc['smooth_lambda'] = 0
df = pd.DataFrame([smooth_auroc, patchy_auroc])
df.to_csv(out_dir / 'results.csv')